- [Draft de PAC](https://pad.etalab.studio/gg7b0FNvSb212-f3OTXL3g?both#)

In [2]:
%run "librairies.ipynb"

In [4]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?datagouvid WHERE { ?item wdt:P3206 ?datagouvid . SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr" } }"""

def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

wikidata = pd.DataFrame([ { 'wikidata': result['item']['value'], 'data.gouv.fr': result['datagouvid']['value'] } for result in results["results"]["bindings"] ])

In [6]:
wikidata.count()

data.gouv.fr    358
wikidata        358
dtype: int64

In [7]:
wikidata.head()

,data.gouv.fr,wikidata
0,ville-de-montpellier,http://www.wikidata.org/entity/Q6441
1,region-ile-de-france,http://www.wikidata.org/entity/Q13917
2,ministere-de-la-defense,http://www.wikidata.org/entity/Q75706
3,ministere-des-armees,http://www.wikidata.org/entity/Q75706
4,le-parisien,http://www.wikidata.org/entity/Q142348


In [9]:
organisations = pd.read_csv("data/organizations.csv", sep=";")

In [13]:
organisations.head()

,Unnamed: 0,_cls,_id,acronym,badges,created_at,description,ext,extras,image_url,last_modified,logo,members,metrics,name,requests,slug,teams,url,zone
0,0,Organization,534fffa3a3a7292c64a78092,NaN,NaN,2014-04-17 18:21:55.187,Le Pays de Morlaix est composé de 4 groupement...,{},{'datagouv_ckan_id': 'fd367dca-d0be-43be-bc42-...,http://static.data.gouv.fr/50/a7688f98d2765a3f...,2018-08-30 14:46:54.156,NaN,[{'user': ObjectId('534fff4aa3a7292c64a77c2e')...,"{'datasets': 0, 'followers': 1, 'members': 1, ...",Pays de Morlaix,NaN,pays-de-morlaix,[],NaN,NaN
1,1,Organization,5534f184c751df3bce37dddb,NaN,NaN,2015-04-20 14:31:00.758,Société spécialisée dans le conseil et l’évalu...,{},{},NaN,2018-12-07 08:02:49.823,{'original': '2015-04-20/a5e47a6b6b684345bfa14...,[{'user': ObjectId('5534bd2cc751df6ebb37dddb')...,"{'nb_uniq_visitors': 0, 'datasets': 0, 'nb_vis...",EDATER,[],edater,[],http://www.edater.fr,NaN
2,2,Organization,53ca2a2ea3a7294a1ddd783f,NaN,NaN,2014-07-19 10:19:58.879,Cabinet de Conseil sur les problématiques Cnil...,{},{'datagouv_ckan_id': '96a6ecc1-d964-4be2-ad42-...,http://www.cabinet-cilex.com/uploads/images/lo...,2018-05-30 20:48:13.784,{'original': '4b/3208357e2e40d1a6269ab8e187aa0...,[{'user': ObjectId('53ca29ffa3a7294a1ddd7838')...,"{'datasets': 0, 'followers': 2, 'reuses': 0, '...",Cabinet Cilex,[],cabinet-cilex,[],NaN,NaN
3,3,Organization,537d591aa3a72973a2dc0271,NaN,NaN,2014-05-22 03:55:38.327,Data expertise,{},{'datagouv_ckan_id': '640b5289-095b-4789-a29b-...,http://i.imgur.com/52NTL9S.png,2017-09-05 03:15:48.168,{'original': '73/3e5b8cc53f4a07a1d2971519b19c6...,[{'user': ObjectId('537d58c5a3a72973a2dc025a')...,"{'datasets': 0, 'followers': 0, 'members': 1, ...",Jalgos,NaN,jalgos,[],NaN,NaN
4,4,Organization,534fffbaa3a7292c64a78163,NaN,NaN,2014-04-17 18:22:18.556,[Voxe.org](http://www.voxe.org) est le compara...,{},{'datagouv_ckan_id': 'b1fc7173-d24b-4bfe-96c0-...,http://opengov.newschallenge.org/open/open-gov...,2018-12-17 13:55:42.869,{'original': 'd9/fba9b7c803472e96416dd9e9ec12d...,[{'user': ObjectId('534fff3fa3a7292c64a7759b')...,"{'datasets': 2, 'followers': 3, 'members': 1, ...",Voxe.org,NaN,voxe-org,[],NaN,NaN


In [14]:
couverture = pd.merge(organisations, wikidata, how='inner', left_on="slug", right_on="data.gouv.fr")

couverture.count()

Unnamed: 0       354
_cls             281
_id              354
acronym          116
badges           337
created_at       354
description      334
ext              354
extras           354
image_url        175
last_modified    354
logo             346
members          345
metrics          354
name             354
requests         292
slug             354
teams            354
url              229
zone               1
data.gouv.fr     354
wikidata         354
dtype: int64

In [ ]:
alt.Chart.